In [1]:
# Basic imports
import torch
import brambox as bb
import numpy as np
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt

# Settings
bb.logger.setConsoleLevel('ERROR')             # Only show error log messages

In [2]:
# Load annotations
anno = bb.io.load('pandas', 'data/annotations.pkl')
det = bb.io.load('pandas', 'data/detections.pkl')
my_class = 'aeroplane'

anno = anno[anno['class_label'] == my_class]
det = det[det['class_label'] == my_class]

print('annotations:')
display(anno.head())
print('detections:')
display(det.head())

annotations:


,image,class_label,id,x_top_left,y_top_left,width,height,occluded,truncated,lost,difficult,ignore
80,VOC2007/JPEGImages/000067,aeroplane,0,44.0,73.201923,407.0,139.0,0.0,0.0,False,False,False
223,VOC2007/JPEGImages/000128,aeroplane,0,53.0,117.000000,406.0,115.0,0.0,0.0,False,False,False
361,VOC2007/JPEGImages/000216,aeroplane,0,234.0,170.000000,51.0,55.0,0.0,0.0,False,False,False
411,VOC2007/JPEGImages/000243,aeroplane,0,181.0,127.000000,94.0,67.0,0.0,0.0,False,False,False
429,VOC2007/JPEGImages/000260,aeroplane,0,55.0,50.000000,402.0,186.0,0.0,0.0,False,False,False


detections:


,image,class_label,id,x_top_left,y_top_left,width,height,confidence
1522,VOC2007/JPEGImages/000055,aeroplane,0,293.686830,57.723568,368.280337,141.727686,0.001489
1577,VOC2007/JPEGImages/000056,aeroplane,0,381.949645,218.251283,117.308681,116.749947,0.001485
1926,VOC2007/JPEGImages/000067,aeroplane,0,274.033070,129.546899,35.472106,22.948121,0.001328
1927,VOC2007/JPEGImages/000067,aeroplane,0,164.472066,92.803332,248.021878,85.397500,0.001214
1931,VOC2007/JPEGImages/000067,aeroplane,0,-178.237933,62.843928,396.967704,145.495846,0.001194


In [3]:
pr = bb.stat.pr(det, anno, 0.7)  # IoU threshold of 0.5
ap = bb.stat.ap(pr)

# Find detection threshold with maximal F1
f1 = bb.stat.fscore(pr)
threshold = bb.stat.peak(f1)

In [4]:
# Filter detections based on threshold (computed in previous step from F1-curve)
filtered_det = det[det.confidence >= threshold.confidence].copy()

# Compute TP,FP
tpfp_det = bb.stat.match_det(filtered_det, anno, 0.5)
# match_det() returns detections with a 'tp' and 'fp' column.
print('TP FP:')
display(tpfp_det.head())
display(tpfp_det[tpfp_det['fp']].sort_index())

# Compute FN
fn_anno = bb.stat.match_anno(filtered_det, anno, 0.5)
# match_anno() returns annotations with a 'detection' column that has the index of the matched detection
# We use this to filter unmatched annotations (NaN as index)
fn_anno = fn_anno[fn_anno.detection.isnull()]
print('FN:')
display(fn_anno.head())

TP FP:


,image,class_label,id,x_top_left,y_top_left,width,height,confidence,tp,fp
258736,VOC2007/JPEGImages/006946,aeroplane,0,17.248722,25.081112,400.631171,285.836422,0.975978,True,False
117227,VOC2007/JPEGImages/003160,aeroplane,0,-1.134799,121.519070,493.948240,215.537108,0.972886,True,False
284281,VOC2007/JPEGImages/007632,aeroplane,0,-8.549672,133.196922,510.801792,183.493174,0.969386,True,False
78473,VOC2007/JPEGImages/002110,aeroplane,0,33.530474,56.294808,428.544888,180.666043,0.965919,True,False
71617,VOC2007/JPEGImages/001912,aeroplane,0,1.054507,60.346356,481.622769,191.441958,0.963206,True,False


,image,class_label,id,x_top_left,y_top_left,width,height,confidence,tp,fp
31478,VOC2007/JPEGImages/000846,aeroplane,0,162.731171,113.047783,41.144825,33.067279,0.599210,False,True
40772,VOC2007/JPEGImages/001099,aeroplane,0,-13.508522,98.106568,456.841065,96.914787,0.573860,False,True
167662,VOC2007/JPEGImages/004477,aeroplane,0,195.834618,194.649109,107.252341,34.893169,0.571187,False,True
169133,VOC2007/JPEGImages/004513,aeroplane,0,236.017997,165.885448,26.290866,17.320273,0.551612,False,True
191498,VOC2007/JPEGImages/005106,aeroplane,0,160.729610,203.227703,94.271045,40.235249,0.735629,False,True
227000,VOC2007/JPEGImages/006051,aeroplane,0,158.676588,165.057586,44.783413,23.789690,0.668129,False,True
239509,VOC2007/JPEGImages/006406,aeroplane,0,85.895190,156.243104,90.410443,49.349198,0.693043,False,True
239510,VOC2007/JPEGImages/006406,aeroplane,0,87.681358,184.420549,83.780105,45.678950,0.584113,False,True
279013,VOC2007/JPEGImages/007502,aeroplane,0,212.801897,173.160975,58.243586,48.110403,0.583209,False,True
316152,VOC2007/JPEGImages/008459,aeroplane,0,327.624028,149.840722,163.335287,45.409491,0.674741,False,True


FN:


,image,class_label,id,x_top_left,y_top_left,width,height,occluded,truncated,lost,difficult,ignore,detection,criteria
223,VOC2007/JPEGImages/000128,aeroplane,0,53.0,117.000000,406.0,115.0,0.0,0.0,False,False,False,NaN,0.0
945,VOC2007/JPEGImages/000600,aeroplane,0,1.0,11.000000,450.0,312.0,1.0,0.0,False,False,False,NaN,0.0
975,VOC2007/JPEGImages/000618,aeroplane,0,1.0,36.201923,150.0,178.0,1.0,0.0,False,False,False,NaN,0.0
1103,VOC2007/JPEGImages/000706,aeroplane,0,48.0,285.201923,64.0,47.0,0.0,0.0,False,False,False,NaN,0.0
1104,VOC2007/JPEGImages/000706,aeroplane,0,224.0,188.201923,64.0,49.0,0.0,0.0,False,False,False,NaN,0.0
